**Rio de Janeiro, 27 de Junho de 2016**

Grupo: **Carlos Mattoso** (🎂) _[ **1210553** ]_ e **Gabriel Siqueira** _[ **1210689** ]_

Disciplina: **INF1335 - Introdução a Data Science com R e Python**

# A5 - Rede de Citações

## _Setup_ Básico

In [118]:
options(repr.plot.width = 10)
options(repr.plot.height = 5)

load.lib <- function(lib.name, rep="https://cloud.r-project.org/") {
    if (!require(lib.name, character.only=T))
        install.packages(lib.name, repos=rep)
    library(lib.name, character.only=T)
}
load.lib("knitr")
load.lib("ggplot2")

printf <- function(...) invisible(cat(paste(sprintf(...), "\n", sep=""))) # source: http://stackoverflow.com/a/13023329
# Usage example: print("%s %d", Number, 5)

dir.create("data", showWarnings = FALSE)

# Baixa os dados da pasta HCI do github para uma pasta local `data`
github.base.url = "https://raw.githubusercontent.com/calmattoso/INF1335/master/A5/ihc/" 
file.names = c("artigo", "autor", "instituicao", "keyword", "pessoa", "referencia")
for (file.name in file.names) {
    file.name.extension = paste(file.name, ".csv", sep="")
    file.path = paste("data/", file.name.extension, sep="")
    if (!file.exists(file.path))    
        download.file(paste(github.base.url, file.name.extension, sep=""), destfile=file.path, method="curl")
}
 
# Show user message indicating that different results might be observed if R version is less than 3.2.2
r.version = R.Version()
if (r.version$major < "3") {
    print("This notebook was develop under R major version 3. If your machine runs a version below 3, abnormal behavior should be expected.")
} else if (paste(r.version$major, r.version$minor, sep="") < "3.2.2") {
    print("This notebook was developed under R version 3.2.2.  If your machine runs a version below this, small differences could occur.")
}

## Leitura Inicial dos Dados

In [114]:
data.map = list()
for (file.name in file.names) {
    file.name.extension = paste(file.name, ".csv", sep="")
    file.path = paste("data/", file.name.extension, sep="")
    
    # Read strings as strings due to the nature of the files. This can be fixed on a case by case basis later
    if (file.exists(file.path))    
        data.map[[file.name]] = read.csv(file.path, fileEncoding="UTF-8", stringsAsFactor=F)
}
summary(data.map)

            Length Class      Mode
artigo      8      data.frame list
autor       7      data.frame list
instituicao 6      data.frame list
keyword     3      data.frame list
pessoa      3      data.frame list
referencia  9      data.frame list

[1] "number 5"


## Examinação dos Dados

### _artigo.csv_

Utilizemos as funções _str_ e _summary_ para entender a forma dos dados lidos e se seus tipos são apropriados. Além disso, exibem-se algumas estatísticas sobre atributos textuais, a fim de determinar o melhor tipo para representá-los e detecção de valores faltantes (_missing values_).

In [144]:
artigo = data.map[["artigo"]]

str(artigo)
summary(artigo)

head(artigo)
tail(artigo)

explore.text.attr <- function(attr.data, attr.name) {
    printf("Análise Exploratória de `%s`", attr.name)
    printf("  Número de valores distintos e total: %d/%d", 
           nrow(unique(attr.data)), nrow(attr.data))
    
    printf("  Quantas instâncias tem conteúdo não vazio (não é `string` vazio): %d (%%%.2f)", 
           sum(attr.data != ""), sum(attr.data != "")/nrow(attr.data)*100.0)
    
    random.attr.sample = sample(unique(attr.data[attr.data != ""]), 3)
    printf("  Amostra de instâncias de `%s`:", attr.name)
    printf("    %s", random.attr.sample)
    printf("")
}

# Exploração dos atributos de tipo "char" da base de dados
for (attr.name in names(artigo)) {
    explore.text.attr(artigo[attr.name], attr.name)
}

'data.frame':	340 obs. of  8 variables:
 $ id_artigo    : int  4506 4507 4508 4509 4510 4511 4512 4513 4514 4515 ...
 $ ano          : int  1998 1998 1998 1998 1998 1998 1998 1998 1998 1998 ...
 $ titulo       : chr  "Uma Abordagem Semiótica à Análise de Interfaces: um estudo de caso" "Knowledge and Communication Perspectives in Extensible Applications" "Uma Abordagem Ergonômica para o Desenvolvimento de Sistemas Interativos" "Estudo de Interação para um Sistema de Ensino à Distância Baseado em Interfaces de Realidade Virtual" ...
 $ idioma       : chr  "PT-BR" "EN" "PT-BR" "PT-BR" ...
 $ tema         : chr  "" "" "" "" ...
 $ acm_chave    : chr  "" "" "" "" ...
 $ acm_categoria: chr  "" "" "" "" ...
 $ acm_termos   : chr  "" "" "" "" ...


   id_artigo         ano          titulo             idioma         
 Min.   :4506   Min.   :1998   Length:340         Length:340        
 1st Qu.:4591   1st Qu.:2002   Class :character   Class :character  
 Median :4676   Median :2010   Mode  :character   Mode  :character  
 Mean   :4677   Mean   :2008                                        
 3rd Qu.:4760   3rd Qu.:2013                                        
 Max.   :4854   Max.   :2015                                        
     tema            acm_chave         acm_categoria       acm_termos       
 Length:340         Length:340         Length:340         Length:340        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                       

,id_artigo,ano,titulo,idioma,tema,acm_chave,acm_categoria,acm_termos
1,4506,1998,Uma Abordagem Semiótica à Análise de Interfaces: um estudo de caso,PT-BR,,,,
2,4507,1998,Knowledge and Communication Perspectives in Extensible Applications,EN,,,,
3,4508,1998,Uma Abordagem Ergonômica para o Desenvolvimento de Sistemas Interativos,PT-BR,,,,
4,4509,1998,Estudo de Interação para um Sistema de Ensino à Distância Baseado em Interfaces de Realidade Virtual,PT-BR,,,,
5,4510,1998,APIS: Método para Desenvolvimento de Interfaces Homem-Computador em Sistemas de Segurança Visando a Confiabilidade Humana,PT-BR,,,,
6,4511,1998,"Interação, Interfaces e Instrumentos em Música Eletroacústica",PT-BR,,,,


,id_artigo,ano,titulo,idioma,tema,acm_chave,acm_categoria,acm_termos
335,4849,2015,O Pensamento Computacional Através de Jogos Infantis,PT-BR,education and hci,H.5.2. User Interfaces.,,
336,4850,2015,Investigating Treemap Visualization in Inverted Scale,EN,information visualization,H.5.2. User Interfaces: Evaluation/Methodology,,
337,4851,2015,Visualization Mechanisms for Crowdsourcing Information in Emergency Coordination,EN,information visualization,H.4.2 Information system applications: Types of Systems— decision support,,
338,4852,2015,Eye Jam: A Gaze-Controlled Musical Interface,EN,interaction with new devices,"H.5.2 User Interfaces: Interaction styles, theory and methods, I.3.6 Methodology and Techniques: Int",,
339,4853,2015,Identifying Cognitive Authority in Social Networks: a conceptual framework,EN,social interaction,H.5 Information Interfaces and Presentation: General,,
340,4854,2015,Interaction Design Process for Healthcare Professionals: formalizing user's contexts observations,EN,design and evaluation,H.5.2. User interface,,


Análise Exploratória de `id_artigo`
  Número de valores distintos e total: 340/340
  Quantas instâncias tem conteúdo não vazio (não é `string` vazio): 340 (%100.00)
  Amostra de instâncias de `id_artigo`:
    4696
     4578
     4569

Análise Exploratória de `ano`
  Número de valores distintos e total: 14/340
  Quantas instâncias tem conteúdo não vazio (não é `string` vazio): 340 (%100.00)
  Amostra de instâncias de `ano`:
    2010
     2012
     2014

Análise Exploratória de `titulo`
  Número de valores distintos e total: 340/340
  Quantas instâncias tem conteúdo não vazio (não é `string` vazio): 340 (%100.00)
  Amostra de instâncias de `titulo`:
    Using and utilizing an innovative media development tool
     Integração do Kinect com OpenCV para tratar interações via gestos
     Design da Informação para Sistemas de Realidade Mediada

Análise Exploratória de `idioma`
  Número de valores distintos e total: 5/340
  Quantas instâncias tem conteúdo não vazio (não é `string` vazio): 340 

Observa-se que os atributos _acm_ tem bastante dados faltantes, pecando a base então em capturar sua informação. Contudo, seu tipo ser _character_ faz sentido. Por outro lado, é o atributo _idioma_ define um conjunto restrito de línguas, sendo então razoável atualizar seu tipo para _factor_. Os demais atributos não apresentam valores faltantes, o que é sempre agradável de se ver.

In [147]:
artigo$idioma = as.factor(artigo$idioma)
summary(artigo$idioma)

EN    ES  P-BR PT-BR PT-PT 
  108     2     1   228     1

Aqui já observa-se uma oportunidade de limpar nossos dados. Existem duas classificações para o idioma português do Brasil, `P-BR` e `PT-BR`, o que deve ser um erro. Há ainda uma instância discriminada a português de Portugual (`PT-PT`). Optamos por fundir estes três tipos simplesmente como `PT`, seguindo a forma das demais classificações de idiomas da base, `EN` para inglês e `ES` para espanhol.

In [153]:
artigo$idioma = as.character(artigo$idioma)
artigo$idioma[artigo$idioma == "P-BR" | artigo$idioma == "PT-PT" | artigo$idioma == "PT-BR"] = "PT"
artigo$idioma = as.factor(artigo$idioma)
summary(artigo$idioma)

EN  ES  PT 
108   2 230

Apresenta-se abaixo o _summary_ e _str_ da base manipulada:

In [155]:
data.map[["artigo"]] = artigo
summary(data.map[["artigo"]])
str(data.map[["artigo"]])

   id_artigo         ano          titulo          idioma       tema          
 Min.   :4506   Min.   :1998   Length:340         EN:108   Length:340        
 1st Qu.:4591   1st Qu.:2002   Class :character   ES:  2   Class :character  
 Median :4676   Median :2010   Mode  :character   PT:230   Mode  :character  
 Mean   :4677   Mean   :2008                                                 
 3rd Qu.:4760   3rd Qu.:2013                                                 
 Max.   :4854   Max.   :2015                                                 
  acm_chave         acm_categoria       acm_termos       
 Length:340         Length:340         Length:340        
 Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character  
                                                         
                                                         
                                                         

'data.frame':	340 obs. of  8 variables:
 $ id_artigo    : int  4506 4507 4508 4509 4510 4511 4512 4513 4514 4515 ...
 $ ano          : int  1998 1998 1998 1998 1998 1998 1998 1998 1998 1998 ...
 $ titulo       : chr  "Uma Abordagem Semiótica à Análise de Interfaces: um estudo de caso" "Knowledge and Communication Perspectives in Extensible Applications" "Uma Abordagem Ergonômica para o Desenvolvimento de Sistemas Interativos" "Estudo de Interação para um Sistema de Ensino à Distância Baseado em Interfaces de Realidade Virtual" ...
 $ idioma       : Factor w/ 3 levels "EN","ES","PT": 3 1 3 3 3 3 3 3 3 3 ...
 $ tema         : chr  "" "" "" "" ...
 $ acm_chave    : chr  "" "" "" "" ...
 $ acm_categoria: chr  "" "" "" "" ...
 $ acm_termos   : chr  "" "" "" "" ...


### _autor.csv_ 


### _instituicao.csv_ 

### _keyword.csv_ 

### _pessoa.csv_ 

### _referencia.csv_